In [35]:
import pandas as pd, numpy as np
from influxdb import DataFrameClient
import json
%cd /home/jovyan/work

/home/jovyan/work


Set params

In [36]:
purge=False
# purge=True

In [37]:
process_annoations=False
# process_annoations=True

In [38]:
# process_macro=False
process_macro=True

In [39]:
# write_out=False
write_out=True

In [40]:
full_overwrite=False
# full_overwrite=True

Load pre-params

In [41]:
df0=pd.read_csv('df0.csv').set_index('date')
df0.index=pd.to_datetime(df0.index)

In [42]:
bets=json.loads(open('bets.json','r').read())

In [43]:
severity=pd.read_csv('severity.csv').set_index('date')
severity.index=pd.to_datetime(severity.index)

In [44]:
mobility=pd.read_csv('mobility.csv').set_index('date')
mobility.index=pd.to_datetime(mobility.index)

In [45]:
mobility_mini=pd.read_csv('mobility_mini.csv').set_index('date')
mobility_mini.index=pd.to_datetime(mobility_mini.index)

Establish DB connection

In [46]:
user = 'admin'
password = open('auth/influxa.txt','r').read()
host='influxdb'
port=8086
dbname='base'
dbname_long='long'
protocol = 'line' #'json'
client = DataFrameClient(host, port, user, password, dbname)
client_long = DataFrameClient(host, port, user, password, dbname_long)

In [47]:
if purge:
    client.drop_database(dbname)
    client.drop_retention_policy(dbname)
    client.create_database(dbname)
    client.create_retention_policy(dbname, '600d', 1, default=True)
    client_long.drop_database(dbname_long)
    client_long.drop_retention_policy(dbname_long)
    client_long.create_database(dbname_long)
    client_long.create_retention_policy(dbname_long, '6000d', 1, default=True)

In [48]:
htmlipath='../html/'
htmlepath='//myserv-html.er/'
htmlepath_other='//mybackupserv-html.er/'

In [49]:
titles={'HU':"Magyar",'RO':'Română','EN':'English'}

In [50]:
rtitles={titles[t]:t for t in titles}

In [51]:
import requests
grafana = "http://grafana:3000/"
headers = {
    'Authorization': 'Bearer '+open('auth/grafana.txt','r').read(),
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [52]:
response = requests.get(grafana+'api/folders', headers=headers)
folders=json.loads(response.content)

In [53]:
folder_id=[f['id'] for f in folders if f['title']=='My Grafana Folder'][0] #General

In [54]:
response = requests.get(grafana+'api/search?folderIds='+str(folder_id), headers=headers)
dashs=json.loads(response.content)

In [55]:
uids={rtitles[d['title']]:d['uid'] for d in dashs if d['title'] in rtitles}
iids={rtitles[d['title']]:d['id'] for d in dashs if d['title'] in rtitles}

In [56]:
uids_light={rtitles[d['title'].split(' Light')[0]]:d['uid'] for d in dashs if 'Light' in d['title']}
iids_light={rtitles[d['title'].split(' Light')[0]]:d['id'] for d in dashs if 'Light' in d['title']}

In [58]:
languages=['HU','RO','EN']

In [59]:
url='https://docs.google.com/spreadsheets/d/'+open('auth/sheet.txt','r').read()+'/gviz/tq?tqx=out:csv&sheet='

In [60]:
def get_szotar():
    sheet='szotar'
    columns=languages+[i+'_description' for i in languages]+[i+'_source' for i in languages]
    df=pd.read_csv(url+sheet)
    df=df[['ID','UI']+columns]
    sheet='minidashboard'
    df2=pd.read_csv(url+sheet)
    df2=df2[['ID','UI']+columns]
    df=pd.concat([df,df2])
    szotardf=df.set_index('ID')[columns]
    szotar=df.set_index('ID').T.to_dict()
    szotarHU=df.set_index('HU',drop=False).T.to_dict()
    szotarRO=df.set_index('RO',drop=False).T.to_dict()
    szotarEN=df.set_index('EN',drop=False).T.to_dict()
    return szotardf,szotar,szotarHU,szotarRO,szotarEN

In [61]:
def resolve_time_conflicts(series):
    ds={}
    ts=[]
    for d in series:
        if d not in ds:
            ds[d]=pd.to_datetime(d)
            t=(ds[d])
        else:
            ds[d]=ds[d]+pd.to_timedelta('193m')
            t=(ds[d])
        ts.append(t)
    return ts

# Influx

In [62]:
szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

<ipython-input-60-77652dbc15c9>:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarHU=df.set_index('HU',drop=False).T.to_dict()
<ipython-input-60-77652dbc15c9>:13: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarRO=df.set_index('RO',drop=False).T.to_dict()
<ipython-input-60-77652dbc15c9>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  szotarEN=df.set_index('EN',drop=False).T.to_dict()


In [63]:
import time
import datetime
import pytz
utc=pytz.UTC

In [64]:
def push2influx(df,measurement,field_columns,tag_columns,shift=False,dbclient=client,wo=write_out,fo=full_overwrite,daily=True):
    if wo:
        df=df.sort_index()
        # df.index = utc.localize(df.index) 
        df.index=df.index.tz_localize('GMT')#.tz_convert('Europe/Bucharest')
        if shift:
            df.index+=pd.to_timedelta('12h')
        if fo: 
            print('Purging',measurement,'...')
            dbclient.query('DROP MEASUREMENT '+measurement)
        else:
            latest=dbclient.query('SELECT * FROM '+measurement+' GROUP BY "1d" ORDER BY DESC LIMIT 1')
            if latest:
                lat=latest[list(latest.keys())[0]].index[0]
                if daily: lat+=pd.to_timedelta('1d')
                df=df[lat:]
                print('Slicing',measurement,'from',lat,'...')
            else:
                print('No data in db for',measurement,'...')
        time.sleep(3)
        print('Writing to',measurement,'...')
        bsize=5000
        bwait=2
        print(len(df),'data points will be written in',len(df)/bsize,'batches.')
        print('Expected query running time is:',int((len(df)/bsize)*bwait*1.1)+3,'seconds.')
        for i in range(int(len(df)/bsize)+1):
            r=range(i*bsize,min(len(df),(i+1)*bsize))
            dc=df.iloc[r]
            print('Writing batch',i+1,'...')
            dbclient.write_points(dc, measurement, protocol=protocol,
                                field_columns=field_columns,
                                tag_columns=[])
            time.sleep(bwait)
        time.sleep(3)
        print('Done!')
    else:
        print('Write-out not enabled. Skipping...')

DateLaZi

In [65]:
#new method from DateLaZi
# dlz=requests.get('https://datelazi.ro/latestData.json').content
# dlz=requests.get('https://di5ds1eotmbx1.cloudfront.net/latestData.json').content
dlz=requests.get('https://d35p9e4fm9h3wo.cloudfront.net/latestData.json').content
dlz=json.loads(dlz)
dlz['historicalData'][dlz['currentDayStats']['parsedOnString']]=dlz['currentDayStats']
dlzs=[]
dlz_counties=[]
for date in dlz['historicalData']:
    d={}
    mdate=date.replace('2018-11-07','2020-11-07').replace('2020-01-05','2021-01-05').replace('2020-01-07','2021-01-07')
    d['date']=mdate
    d['cases']=dlz['historicalData'][date]['numberInfected']
    d['heals']=dlz['historicalData'][date]['numberCured']
    d['deaths']=dlz['historicalData'][date]['numberDeceased']
    if 'vaccines' in dlz['historicalData'][date]:
        if dlz['historicalData'][date]['vaccines']:
            for v in dlz['historicalData'][date]['vaccines']:
                for k in ['total_administered','immunized']:
                    k2=k.replace('immunized','total_immunized')
                    if k2 not in d: d[k2]=0
                    d[k2]+=dlz['historicalData'][date]['vaccines'][v][k]
                    d[k2+'_'+v]=dlz['historicalData'][date]['vaccines'][v][k]
    if 'countyInfectionsNumbers' in dlz['historicalData'][date]:
        counties=dlz['historicalData'][date]['countyInfectionsNumbers']
        if counties:
            for county in counties:
                dummy={'date':mdate,county:dlz['historicalData'][date]['countyInfectionsNumbers'][county]}
                if county=='PH':
                    if date=='2020-12-01':
                        dummy[county]=19395
                dlz_counties.append(dummy)
    dlzs.append(d)
dl=pd.DataFrame(dlzs).set_index('date').sort_index(ascending=True)

In [66]:
#Manual fixes
dl.loc['2021-04-08','cases']=993613
dl.loc['2021-04-08','heals']=890528
dl.loc['2021-04-08','deaths']=24733

In [67]:
for c in dl.columns[3:]:
    dl[c]=dl[c].cumsum()
dl2=dl.diff()
dl2.columns=['case','heal','death']+[i.replace('total_','') for i in dl2.columns[3:]]
dl['active']=dl['cases']-dl['heals']-dl['deaths']
dl=dl.join(dl2)

In [68]:
latest=dl.index[-1]
now=str(pd.to_datetime("now"))[:10]
print(latest,now)

2021-05-02 2021-05-02


Tests from `graphs.ro`

In [69]:
r=requests.get('https://www.graphs.ro/json.php')
graphs_ro=json.loads(r.content)['covid_romania']
g=pd.DataFrame(graphs_ro)
# g['date']=pd.to_datetime(g['reporting_date'])
g['date']=g['reporting_date']
g=g.set_index('date')
g['tests']=g['total_tests']
g['test']=g['new_tests_today']
latest2=g.index[0]
print(latest2)

2021-05-02


In [70]:
latest=str(min(pd.to_datetime(latest),pd.to_datetime(latest2)))[:10]
print(latest)

2021-05-02


In [71]:
dl=dl.join(g[['tests','test']])

Governance

In [72]:
sheet='governance'

In [73]:
df=pd.read_csv(url+sheet)
# dl=pd.concat([df.set_index('date').sort_index()[:'2020-03-17'][dl.columns[:-1]],
#           dl['2020-03-18':]])
df=df.set_index('date').sort_index(ascending='True').join(dl,lsuffix='_g')#[20:] #[:-1] vaccines nincs a governance-ban

In [74]:
# fill from local, if missing in DateLaZi
for c in ['active','cases','heals','deaths','case','heal','death']:
    df[c]=df[c].fillna(df[c+'_g'].astype(str).str.replace(',','')).replace('nan',np.nan)
df=df.dropna(how='all',axis=0).dropna(how='all',axis=1)

In [75]:
df14=df[['cases']].reset_index().join(df[['cases']][14:].reset_index(),lsuffix='1')
df14['case14']=df14['cases'].astype(float)-df14['cases1'].astype(float)
df=df.join(df14.set_index('date')['case14'].dropna().astype(int))

In [76]:
df.index=pd.to_datetime(df.index)
df0=df.copy()
#df['date']=pd.to_datetime(df['date'])
#df=df[df.columns[:13]].set_index('date')
vaccine_totals=[i for i in list(dl2.columns[3:])+list(dl.columns[3:len(dl2.columns)]) if 'total_' in i]
df=pd.DataFrame(df[['active','cases','heals','deaths','case14']+\
                  vaccine_totals])
for c in [i for i in df.columns if '_administered' in i]:
    df[c.replace('_administered','_first')]=df[c]-df[c.replace('_administered','_immunized')]           
df=df.stack().reset_index().set_index('date')
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').astype(float).astype(int)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']
current=int(df[df['type']=='case14']['value'][-1]) #sajat becsles
# current=df[df['type']=='active']['value'][-1] #official
current

28351

In [77]:
df0.to_csv('df0.csv')

In [78]:
#sanity check
cols=[ 'active','case14', 'case', 'heal', 'death','cases', 'heals', 'deaths']
dx=df0[cols].dropna(how='all')
dx=dx.loc[~dx.index.isnull()]
dx[cols].tail()

,active,case14,case,heal,death,cases,heals,deaths
date,,,,,,,,
2021-04-28,40906.0,35330.0,2240.0,3595.0,150.0,1051779.0,983040.0,27833.0
2021-04-29,39831.0,33328.0,1850.0,2787.0,138.0,1053629.0,985827.0,27971.0
2021-04-30,37763.0,31700.0,1636.0,3566.0,138.0,1055265.0,989393.0,28109.0
2021-05-01,35773.0,29533.0,1307.0,3212.0,85.0,1056572.0,992605.0,28194.0
2021-05-02,33867.0,28351.0,1083.0,2901.0,88.0,1057655.0,995506.0,28282.0


In [84]:
df1=df0[['cases', 'heals', 'deaths', 'total_administered',
   'total_administered_pfizer', 'total_immunized',
   'total_immunized_pfizer', 'total_administered_moderna',
   'total_immunized_moderna', 'total_administered_astra_zeneca',
   'total_immunized_astra_zeneca', 'active', 'case', 'heal', 'death',
   'administered', 'administered_pfizer', 'immunized', 'immunized_pfizer',
   'administered_moderna', 'immunized_moderna',
   'administered_astra_zeneca', 'immunized_astra_zeneca', 'tests', 'test',
   'case14']]

In [85]:
for c in df1:
    df1[c]=df1[c].astype(str).str.replace(',','').astype(float)
df1.to_csv('github/data/time_series_ro_daily.csv')

<ipython-input-85-4446d7339660>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[c]=df1[c].astype(str).str.replace(',','').astype(float)


-> Run till here to test Grafana only.

In [86]:
df1=df.replace(0,np.nan).dropna()
df1=df1[df1['type'].isin(['active', 'cases', 'heals', 'deaths', 'case14'])]
df2=df.replace(0,np.nan).dropna()
df2=df2[~(df2['type'].isin(['active', 'cases', 'heals', 'deaths', 'case14']))]

In [87]:
field_columns=['value']
tag_columns=['type','lang','langtype']
measurement='governance1'
# push2influx(df1,measurement,field_columns,tag_columns,fo=True)
push2influx(df1,measurement,field_columns,tag_columns)
measurement='vaccine1'
# push2influx(df2,measurement,field_columns,tag_columns,fo=True)
push2influx(df2,measurement,field_columns,tag_columns)

Slicing governance1 from 2021-05-03 00:00:00+00:00 ...
Writing to governance1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!
Slicing vaccine1 from 2021-05-03 00:00:00+00:00 ...
Writing to vaccine1 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


In [88]:
# df=pd.read_csv(url+sheet)
df=df0.copy()

# df['date']=pd.to_datetime(df['date'])
# df=df[df.columns[:13]].set_index('date')

Daily cases

In [89]:
df['case/test']=100*df['case'].astype(str).str.replace(',','').astype(float)/\
    df['test'].astype(str).str.replace(',','').astype(float)
df['death_rate']=100*df['death'].astype(str).str.replace(',','').astype(float)/\
    df['case'].astype(str).str.replace(',','').astype(float)

In [90]:
df=pd.DataFrame(df[['death','heal','case','case/test','death_rate']].stack()).reset_index().set_index('date')
df.columns=['type','value']
df['value']=df['value'].astype(str).str.replace(',','').astype(float)
df=df.join(szotardf,on='type')
df=pd.DataFrame(df.reset_index().set_index(['date','type','value']).stack()).reset_index().set_index('date')
df.columns=['type','value','lang','langtype']

In [91]:
field_columns=['value']
tag_columns=['type','lang','langtype']
measurement='governance2'
# push2influx(df.replace(0,np.nan).dropna(),measurement,field_columns,tag_columns,fo=True)
push2influx(df.replace(0,np.nan).dropna(),measurement,field_columns,tag_columns)

Slicing governance2 from 2021-05-03 00:00:00+00:00 ...
Writing to governance2 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Small dashboard

In [94]:
sheet='minidashboard'
dm=pd.read_csv(url+sheet)
# sheet='governance'
# df=pd.read_csv(url+sheet)
# df['date']=pd.to_datetime(df['date'])
# df=df[df.columns[:10]].set_index('date')
# df=df0[list(dl.columns)+['tests','test']].dropna(how='all')
df=df0[list(dl.columns)].dropna(how='all')
df=df.loc[~df.index.isnull()]
for i in dm.set_index('ID')['UI'].iteritems():
    df[i[0]]=i[1]
sheet='EcMonitor'
du=pd.read_csv(url+sheet)
du=du.set_index('date').loc['Unemployment ratio'][2:].dropna()
du.index=[(pd.to_datetime('now')-pd.to_timedelta('11D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(du))][::-1]
du=pd.DataFrame(du)
du.columns=['unemployment']
du.index=pd.to_datetime(du.index)
df=df.join(du)
sheet='employmentdata'
de=pd.read_csv(url+sheet)
de['date']=pd.to_datetime(de['date'])
de=de[de.columns[:3]].set_index('date')
de.columns=['felbontott','felfuggesztett']
# df=df.join(de.ffill())
for c in de.columns:
    de[c]=de[c].astype(str).str.replace(',','').str.replace('%','').astype(float)#.interpolate(method='linear').dropna()
df=df.join(de)
df=df.join(mobility_mini)
for c in df.columns:
    df[c]=df[c].astype(str).str.replace(',','').str.replace('%','').astype(float)
sheet='exchangerates'
dx=pd.read_csv(url+sheet)
dx=dx[['date','EUR - RON (megváltozás)']][5:-1]
dx['date']=pd.to_datetime(dx['date'])
dx=dx.set_index('date')
dx.columns=['xch']
dx['xch']=dx['xch'].astype(str).str.replace(',','').str.replace('%','').astype(float).sort_index().cumsum()
df=df.join(dx)
df['xch']=df['xch'].ffill()
sheet='stocks_all'
dk=pd.read_csv(url+sheet)[bets].dropna()
dk['date']=pd.to_datetime(dk[bets[0]])
dk=dk.set_index('date').drop(bets[0],axis=1)
dk.columns=['bet']
dk=dk.reindex(df.index)
dk['bet']=dk['bet'].astype(str).str.replace(',','').str.replace('%','').astype(float).sort_index().cumsum()
df=df.join(dk)
sheet='MiniGDP'
dg=pd.read_csv(url+sheet)
dg1=dg[['Date','GDP (QoQ, SA)']].dropna()
dg1.index=[(pd.to_datetime('now')-pd.to_timedelta('2D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg1))][::-1]
dg2=dg[['Date.1','GDP']].dropna()
dg2.index=[(pd.to_datetime('now')-pd.to_timedelta('8D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg2))][::-1]
sheet='DFMChartYoY'
dg3=pd.read_csv(url+sheet).T
dg3=pd.read_csv(url+sheet,skiprows=19,nrows=2).T.iloc[1:].dropna()[1]
dg3.index=[(pd.to_datetime('now')-pd.to_timedelta('1D')*(1+i)).strftime('%Y-%m-%d') for i in range(len(dg3))][::-1]
dg=dg1.join(dg2).join(dg3)[['GDP','GDP (QoQ, SA)',1]].astype(float)
dg.columns=['gdp','gdpq','employees']
dg.index=pd.to_datetime(dg.index)
df=df.join(dg)
df=df[:pd.to_datetime('now')]

In [95]:
field_columns=list(df.columns)
tag_columns=[]
measurement='governance3'
# push2influx(df,measurement,field_columns,tag_columns,wo=True,fo=True)
push2influx(df,measurement,field_columns,tag_columns,wo=True)

Slicing governance3 from 2021-05-03 00:00:00+00:00 ...
Writing to governance3 ...
0 data points will be written in 0.0 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


# Grafana

In [96]:
today0=now[-2:]
if today0[0]=='0': 
    today=today0[1]
else:
    today=today0
currents=str(current)[:-3]+'&nbsp;'+str(current)[-3:]
hour=str(pd.to_datetime('now')+pd.to_timedelta('3h'))[11:16]
str(szotar['report']['UI'][1:]),current,currents,today0,hour

('432', 28351, '28&nbsp;351', '02', '01:29')

In [97]:
def darken2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0xfefefe >> 1))[2:]
def brighten2(color):
    return '#'+str(hex(int("0x"+color[1:], 16) & 0x7f7f7f << 1))[2:]
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return mc.rgb2hex(colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2]))

In [98]:
# szotardf,szotar,szotarHU,szotarRO,szotarEN=get_szotar()

In [99]:
colors_to_darken=['#F2CC0C','#CA95E5','#FF780A','#E0B400','#96D98D','#F2495C','#E02F44',
                  'lime','#73BF69','#8AB8FF','#3274D9','#D3D3D3','#FF7383','#FADE2A']

# Maps

In [100]:
top=3

In [101]:
# iso_counties=pd.read_html('https://en.wikipedia.org/wiki/Counties_of_Romania')[1]\
#     .set_index(['ISO code[note 3]'])['County'].to_dict()
# iso_counties['BN']='Bistrița-Năsăud'
# iso_counties['CL']='Călărași'
# open(htmlipath+'panels/iso_counties.json','w').write(json.dumps(iso_counties))
iso_counties=json.loads(open(htmlipath+'panels/iso_counties.json','r').read())

In [102]:
georo=json.loads(open(htmlipath+'panels/romania-counties.json','r').read())
georoco={i['properties']['NAME_1']:i['properties']['ID_1'] for i in georo['objects']['ROU_adm1']['geometries']}   
georoco['București']=georoco['Bucharest']

In [103]:
sheet='industry_county'
df=pd.read_csv(url+sheet)[:-2]
df=df[df.columns[:5]]
df.columns=['HU','RO','EN','q2','q3']
for q in ['q2','q3']:
    df[q]=df[q].str.replace('%','').astype(float)
df=df.set_index('RO').join(pd.DataFrame(georoco,index=['county']).T).reset_index().dropna()
df.columns=['RO','HU','EN','q2','q3','county']

In [104]:
for lang in languages:
    dc=df[[lang,'q2','q3','county']]
    dc.columns=['county','q2','q3','id']
    for c in ['q2','q3']:
        dd=dc.set_index([c])[['county']].sort_index(ascending=False)
        dds=[]
        ddd=dd.iloc[:top]
        ddd[c+'_high']=True
        dds.append(ddd)
        ddd=dd.iloc[-top:]
        ddd[c+'_high']=False
        dds.append(ddd)
        dds=pd.concat(dds)
        dds[c+'_display']=True
        dc=dc.set_index(['county']).join(dds.reset_index().set_index(['county'])\
                                         [[c+'_display',c+'_high']]).reset_index()
        dc[c+'_display']=dc[c+'_display'].fillna(False)
        
    dcqs=[]
    for c in ['q2','q3']:
        dcq=dc.set_index([c,'county'])[[c+'_display',c+'_high']]
        dcq.index.names=['value','county']
        dcq['quarter']=c
        dcq=dcq.reset_index().set_index(['quarter','county','value'])
        dcq.columns=['display','high']
        dcq['high']=dcq['high'].fillna(False)
        dcqs.append(dcq)
    dcqs=pd.concat(dcqs)
    dcc=dc.set_index(['county','id'])[['q2','q3']].stack().reset_index()
    dcc.columns=['county','id','quarter','value']
    dcc=dcc.set_index(['quarter','county','value']).join(dcqs).reset_index()
    dcc['q']=dcc['quarter'].replace({'q2':'Q2 vs Q1','q3':'Q3 vs Q2'})
    dcc['q0']=dcc['quarter'].replace({'q2':2019,'q3':2020})
    # dcc=dcc.sort_values('quarter',ascending=False)
    open(htmlipath+'panels/county_'+lang+'.json','w').write(json.dumps(list(dcc.T.to_dict().values())))
    # open('county_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

<ipython-input-104-1a3a71a98fd4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddd[c+'_high']=True
<ipython-input-104-1a3a71a98fd4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddd[c+'_high']=False


Case county map

In [105]:
sheet='counties_current'
dg=pd.read_csv(url+sheet, header=None)
dgl=dg[[1,4]].set_index(4).T.to_dict()
dgl['Dambovita']=dgl['Dâmbovita']
dgl['Szucsáva']=dgl['Suceava']
dgl['Galac']=dgl['Galati']
dgl['Valcea']=dgl['Vâlcea']
dfl=df[languages+['county']].set_index('HU')
dh=dfl.join(pd.DataFrame(dgl).T).reset_index().dropna()

In [106]:
for lang in languages:
    dc=dh[[lang,1,'county']]
    dc.columns=['county','value','id']
    open(htmlipath+'panels/county2_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))
    # open('county2_'+lang+'.json','w').write(json.dumps(list(dc.T.to_dict().values())))

Case county map fancy

In [107]:
pop=json.loads(open(htmlipath+'panels/pop.json').read())
pop={i:pop[i][1] for i in pop}
pdf=pd.DataFrame(pop,index=['pop']).T
pdf.index.name='iso'

In [108]:
dq=pd.DataFrame(dlz_counties).groupby('date').mean().drop('-',axis=1).stack().reset_index()
dq.columns=['date','iso','cases']
iso2=pd.DataFrame(iso_counties,index=['EN']).T.reset_index()
iso2.columns=['iso','EN']
dq=dq.set_index('iso').join(pdf).join(iso2.set_index('iso')).set_index('EN').join(dh.set_index('EN').drop(1,axis=1)).reset_index()
dq['case_cap']=dq['cases']*1000/dq['pop']

In [109]:
dmss=[]
for lang in languages:
    dms=[]
    # dm=dq.set_index([lang,'date'])[['cases','case_cap','pop','county']]
    dm=dq.set_index('EN').join(iso2.set_index('EN')).reset_index().set_index([lang,'date'])[['cases','case_cap','pop','county','iso']]
    for m in dm.index.get_level_values(0).unique():
        dc=dm.loc[m].sort_index()
        p=dc.iloc[0]['pop']
        dc['case_14']=dc['cases'].diff().rolling(14).sum()
        dc['case_14_cap']=dc['case_14']*1000/p
        dc['id']=dc['county']
        dc['county']=m
        dms.append(dc.dropna())
    dms=pd.concat(dms)
    dms=dms[((dms['case_14_cap']<14)&(dms['case_14_cap']>0))] #filter out non-sense values
    open(htmlipath+'panels/county2c_'+lang+'.json','w').write(json.dumps(list(dms.reset_index().T.to_dict().values())))
    dms2=dms.sort_index()['2020-08-05':]
    open(htmlipath+'panels/county2d_'+lang+'.json','w').write(json.dumps(list(dms2.reset_index().T.to_dict().values())))
    
    dc=dms.reset_index().sort_index(ascending=False)
    for c in ['cases','case_cap','case_14','case_14_cap']:
        dd=dc.reset_index().set_index(['date',c])[['county']].sort_index(ascending=False)
        dds=[]
        for d in dd.index.get_level_values(0).unique():
            dds.append(dd.loc[[d]].iloc[:top])
            dds.append(dd.loc[[d]].iloc[-top:])
        dds=pd.concat(dds)
        dds[c+'_display']=True
        dc=dc.set_index(['date','county']).join(dds.reset_index().set_index(['date','county'])[c+'_display']).reset_index()
        dc[c+'_display']=dc[c+'_display'].fillna(False)
        
    d=dc.iloc[[0]]
    d['county']='d'
    d['case_14_cap']=4
    d['id']=99
        
    open(htmlipath+'panels/county2b_'+lang+'.json','w').write(json.dumps(list(d.T.to_dict().values())+list(dc.T.to_dict().values())))
    open(htmlipath+'panels/daily/county2b_'+latest+'_'+lang+'.json','w').write(json.dumps(list(d.T.to_dict().values())+list(dc.T.to_dict().values())))
    dms['lang']=lang
    dmss.append(dms)
    print(lang)

<ipython-input-109-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-109-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-109-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

HU


<ipython-input-109-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-109-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-109-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

RO


<ipython-input-109-0c1092ac59e8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['county']='d'
<ipython-input-109-0c1092ac59e8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['case_14_cap']=4
<ipython-input-109-0c1092ac59e8>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

EN


In [110]:
dmss[1].to_csv('github/data/time_series_ro_counties_daily.csv')

In [111]:
cases_counties2=open(htmlipath+'panels/cases_counties2.html','r').read()
cases_counties2=cases_counties2[:cases_counties2.find('daily/county2b_\' + \'2021-')+20]+latest+cases_counties2[cases_counties2.find('daily/county2b_\' + \'2021-')+30:]
open(htmlipath+'panels/cases_counties2.html','w').write(cases_counties2)

18156

Counties per day

In [453]:
dz=dh.set_index('EN').join(pd.DataFrame(pd.DataFrame(iso_counties,index=['county'])\
                                     .T.reset_index().set_index('county'))).reset_index()
dlz_data=pd.DataFrame(pd.DataFrame(dlz_counties).groupby('date').mean().stack()).reset_index()
dlz_data.columns=['date','index','value']
dz=dlz_data.set_index('index').join(dz.set_index('index')).dropna().set_index('date')
dz.index=pd.to_datetime(dz.index)
dz2=dz.copy()

In [454]:
# dz2=dz2.ffill()

In [455]:
dz=pd.DataFrame(dz.set_index(['county','value',1],append=True).stack()).reset_index()
dz.columns=['date','county','value','dump','lang','langtype']
dz=dz.set_index('date').drop('dump',axis=1)

In [456]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='counties'
push2influx(dz,measurement,field_columns,tag_columns)

Slicing counties from 2021-05-02 00:00:00+00:00 ...
Writing to counties ...
126 data points will be written in 0.0252 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Counties active

In [457]:
dz2=dz2.reset_index().set_index(['EN','HU','RO','county',1,'date'])\
    .unstack().diff(axis=1).stack().reset_index().set_index('date')

In [458]:
dz2=pd.DataFrame(dz2.set_index(['county','value',1],append=True).stack()).reset_index()
dz2.columns=['date','county','value','dump','lang','langtype']
dz2=dz2.set_index('date').drop('dump',axis=1)

In [459]:
field_columns=['value']
tag_columns=['lang','langtype']
measurement='counties2'
push2influx(dz2,measurement,field_columns,tag_columns)

Slicing counties2 from 2021-05-02 00:00:00+00:00 ...
Writing to counties2 ...
126 data points will be written in 0.0252 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


County intensity map

In [460]:
dmsr=pd.concat(dmss)[['case_14_cap','county','iso','lang']]
dmsr.index=pd.to_datetime(dmsr.index)

In [461]:
field_columns=['case_14_cap']
tag_columns=['county','iso','lang']
measurement='counties3'
push2influx(dmsr,measurement,field_columns,tag_columns)

Slicing counties3 from 2021-05-02 00:00:00+00:00 ...
Writing to counties3 ...
126 data points will be written in 0.0252 batches.
Expected query running time is: 3 seconds.
Writing batch 1 ...
Done!


Push to Grafana

In [462]:
nevuto={'0':'á','1':'é','3':'á','2':'á','4':'é','5':'é','6':'á','7':'é','8':'á','9':'é'}
for lang in languages:
    response = requests.get(grafana+'api/dashboards/uid/'+lang.lower(), headers=headers)
    model=json.loads(response.content)['dashboard']
    for i in model['panels']:
        if lang=='HU':
            #print(i['id'],i['type'],i['title'])
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='EN':
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('</b>')-2]+today0+\
                i['content'][i['content'].rfind('</b>'):]
        elif lang=='RO':
            if i['id']==82:
                i['content']=i['content'][:i['content'].find('<b>')+3]+str(szotar['report']['UI'][1:])+\
                    i['content'][i['content'].find('</b>'):i['content'].rfind('<b>')+3]+currents+\
                    i['content'][i['content'].rfind('</b>'):]
            if i['id']==93:
                i['content']=i['content'][:i['content'].rfind('<b>')+3]+today0+\
                i['content'][i['content'].rfind('<b>')+5:]
            
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })       
    print(r,r.content)
    open(lang+'.json','w').write(json.dumps(model))    
    
    model_light=json.dumps(model).replace('dark','light',9999).replace('lightGray','#52545C',9999)\
        .replace('#d3d3d3','#52545c',9999)\
        .replace('csaladen.es/favicon.ico" style="','csaladen.es/favicon.ico" style="filter: brightness(0.3);',9999)
    for color in colors_to_darken:
        model_light=model_light.replace(color,adjust_lightness(color,0.8),9999)
        model_light=model_light.replace(color.lower(),adjust_lightness(color,0.8),9999)
    model=json.loads(model_light)
    model['title']=titles[lang]+' Light'
    model['uid']=uids_light[lang]
    model['id']=iids_light[lang]
    r=requests.post(grafana+'api/dashboards/db', headers=headers, json={"dashboard":model,
                                                                            "folderId": folder_id,
                                                                            "overwrite": True
                                                                           })    
    print(r,r.content)
    open(lang+'-light.json','w').write(json.dumps(model))   

<Response [200]> b'{"id":2,"slug":"magyar","status":"success","uid":"hu","url":"/d/hu/magyar","version":362}'
<Response [200]> b'{"id":5,"slug":"magyar-light","status":"success","uid":"hu-light","url":"/d/hu-light/magyar-light","version":178}'
<Response [200]> b'{"id":3,"slug":"romana","status":"success","uid":"ro","url":"/d/ro/romana","version":265}'
<Response [200]> b'{"id":6,"slug":"romana-light","status":"success","uid":"ro-light","url":"/d/ro-light/romana-light","version":170}'
<Response [200]> b'{"id":4,"slug":"english","status":"success","uid":"en","url":"/d/en/english","version":226}'
<Response [200]> b'{"id":7,"slug":"english-light","status":"success","uid":"en-light","url":"/d/en-light/english-light","version":171}'


Vaccines per age group

In [120]:
# datagov1a=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/bc19c354-644d-4a24-a26f-512129dbc70d/download/transparenta_vaccinare_martie_2021.xlsx')
# datagov1b=pd.read_excel('https://data.gov.ro/dataset/b86a78a3-7f88-4b53-a94f-015082592466/resource/a0eb9ff2-9b97-430c-b285-360cadb55307/download/vaccinare-covid19-grupe-risc-01-01.04.2021.xlsx')

Configure remote git for `denesdata/roeim/data`

In [ ]:
!git config core.sparseCheckout true

In [122]:
# !mkdir github
%cd /home/jovyan/work/github
# !git init
# !git remote add -f origin https://github.com/denesdata/roeim

/home/jovyan/work/github


In [123]:
# !git config core.sparseCheckout true

In [124]:
# !echo "data" >> .git/info/sparse-checkout

In [125]:
# !git pull origin master

In [21]:
#Might have to go out to Terminal
#Cache GitHub credentials for one month #3600*24*30
# !git config --global credential.helper "cache --timeout=2592000"

In [126]:
%%script env my_bash_variable="$latest2" bash
git add --all
git commit --all -m "automated data update $my_bash_variable"
git push origin master

[master 50f9d5b] automated data update 2021-05-02
 3 files changed, 4177 insertions(+), 3188 deletions(-)


To https://github.com/denesdata/roeim
   5677c09..50f9d5b  master -> master
